In [ ]:
!pip install pandas-ta

In [ ]:
import yfinance as yf


In [ ]:
symbol = "TSLA" # Tesla, Inc
start_date = "2023-01-01"
end_date = "2024-12-31"

In [ ]:
df = yf.download(symbol, start=start_date, end=end_date)

/tmp/ipython-input-959484986.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [ ]:
df.columns

MultiIndex([( 'Close', 'TSLA'),
            (  'High', 'TSLA'),
            (   'Low', 'TSLA'),
            (  'Open', 'TSLA'),
            ('Volume', 'TSLA')],
           names=['Price', 'Ticker'])

In [ ]:
df.columns = ['_'.join(col).strip() for col in df.columns.values]
df.rename(columns={'Close_TSLA': 'Close', 'High_TSLA': 'High', 'Low_TSLA': 'Low', 'Open_TSLA': 'Open', 'Volume_TSLA': 'Volume'}, inplace=True)

print("Flattened and renamed columns:")
display(df.columns)
display(df.head())

Flattened and renamed columns:


Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

,Close,High,Low,Open,Volume
Date,,,,,
2023-01-03,108.099998,118.800003,104.639999,118.470001,231402800
2023-01-04,113.639999,114.589996,107.519997,109.110001,180389000
2023-01-05,110.339996,111.750000,107.160004,110.510002,157986300
2023-01-06,113.059998,114.389999,101.809998,103.000000,220911100
2023-01-09,119.769997,123.519997,117.110001,118.959999,190284000


In [ ]:
df.columns

In [ ]:
df.head()

,Close,High,Low,Open,Volume
Date,,,,,
2023-01-03,108.099998,118.800003,104.639999,118.470001,231402800
2023-01-04,113.639999,114.589996,107.519997,109.110001,180389000
2023-01-05,110.339996,111.750000,107.160004,110.510002,157986300
2023-01-06,113.059998,114.389999,101.809998,103.000000,220911100
2023-01-09,119.769997,123.519997,117.110001,118.959999,190284000


# Creating new features "Moving average" and "RSI"

In [ ]:
import pandas_ta as ta

# 10-period Simple Moving Average (SMA)
df['SMA'] = ta.sma(df['Close'], length=10)

# 14-period Relative Strength Index (RSI)
df['RSI'] = ta.rsi(df['Close'], length=14)

In [ ]:
df.tail()

,Close,High,Low,Open,Volume,SMA,RSI
Date,,,,,,,
2024-12-23,430.600006,434.510010,415.410004,431.000000,72698100,435.092999,62.731818
2024-12-24,462.279999,462.779999,435.140015,435.899994,59551800,441.222000,68.422829
2024-12-26,454.130005,465.329987,451.019989,465.160004,76366400,444.158002,65.645587
2024-12-27,431.660004,450.000000,426.500000,449.519989,82666800,445.514001,58.585163
2024-12-30,417.410004,427.000000,415.750000,419.399994,64941000,443.632001,54.576261


In [ ]:
df.head()

,Close,High,Low,Open,Volume,SMA,RSI
Date,,,,,,,
2023-01-03,108.099998,118.800003,104.639999,118.470001,231402800,NaN,NaN
2023-01-04,113.639999,114.589996,107.519997,109.110001,180389000,NaN,100.000000
2023-01-05,110.339996,111.750000,107.160004,110.510002,157986300,NaN,95.618690
2023-01-06,113.059998,114.389999,101.809998,103.000000,220911100,NaN,95.782703
2023-01-09,119.769997,123.519997,117.110001,118.959999,190284000,NaN,96.164181


In [ ]:
print(df.dtypes)

Close     float64
High      float64
Low       float64
Open      float64
Volume      int64
SMA       float64
RSI       float64
dtype: object


In [ ]:
import numpy as np

cols_price = ['Close', 'High', 'Low', 'Open', "SMA"]
col_volume = "Volume"
col_rsi = "RSI"

# Applying logarithm to volume values
df["Volume_log"] = np.log1p(df[col_volume])

df['RSI_scaled'] = df[col_rsi] / 100.0

# Features entering the model (after transformation)
feature_cols = cols_price + ["Volume_log", "RSI_scaled"]

In [ ]:
df[feature_cols].isna().sum() # RSI_scaled contains a null row
df = df.dropna(subset=feature_cols + ["Close"]).reset_index(drop=True)

# Splitting between train and test

In [ ]:
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size].copy()
test_df = df.iloc[train_size:].copy()

# Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_price = MinMaxScaler(feature_range=(0, 1))
scaler_volume = MinMaxScaler(feature_range=(0, 1))


In [ ]:
train_df[cols_price] = scaler_price.fit_transform(train_df[cols_price])
test_df[cols_price] = scaler_price.transform(test_df[cols_price])

In [ ]:
# We only apply .fit_transform to the train set
train_df[["Volume_log"]] = scaler_volume.fit_transform(train_df[["Volume_log"]])

# We only apply .transform to the test set
test_df[["Volume_log"]] = scaler_volume.transform(test_df[["Volume_log"]])

In [ ]:
# Transforming in numpy arrays

X_train_all = train_df[feature_cols].values
X_test_all = test_df[feature_cols].values

In [ ]:
close_index = cols_price.index("Close")

y_train_all = train_df[cols_price].values[:, close_index]
y_test_all = test_df[cols_price].values[:, close_index]

# Creating sequences

In [ ]:
def create_sequences(X, y, seq_length):
  X_seqs, y_seqs = [], []
  for i in range(len(X) - seq_length):
    X_seqs.append(X[i : i + seq_length])
    y_seqs.append(y[i + seq_length])
  return np.array(X_seqs), np.array(y_seqs)

In [ ]:
seq_length = 20

X_train_seq, y_train = create_sequences(X_train_all, y_train_all, seq_length)
X_test_seq, y_test = create_sequences(X_test_all, y_test_all, seq_length)

# Converting to tensors

In [ ]:
import torch

X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)

In [ ]:
print("NaN em X_train_tensor?", torch.isnan(X_train_tensor).any())
print("NaN em y_train_tensor?", torch.isnan(y_train_tensor).any())

NaN em X_train_tensor? tensor(False)
NaN em y_train_tensor? tensor(False)


# DataLoader

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Preparing the model

In [ ]:
import torch
import torch.nn as nn

class LSTMRegressor(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, dropout):
    super().__init__()
    self.lstm = nn.LSTM(
    input_size=input_size,
    hidden_size = hidden_size,
    num_layers = num_layers,
    batch_first=True, # -> input: (batch, seq, feature)
    dropout = dropout if num_layers > 1 else 0.0
    )

    # Layer to map from hidden -> 1 value (regression)
    self.fc = nn.Linear(hidden_size, 1)

  def forward(self, x):
    """
    x: (batch_size, seq_length, input_size)
    """

    # out: (batch_size, seq_length, hidden_size)
    # h_n: (num_layers, batch_size, hidden_size)
    out, (h_n, c_n) = self.lstm(x)

    # We only get the output from the last timestep
    # out[:, 1, :] => (batch_size, hidden_size)
    last_hidden = out[:, -1, :]

    out = self.fc(last_hidden)

    return out

# Instantiating the model

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device:", device)

input_size = len(feature_cols)
print(input_size)

model = LSTMRegressor(input_size=input_size, hidden_size=64, num_layers=2, dropout=0.0)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 20

for epoch in range(num_epochs):
  model.train()
  epoch_loss = 0.0

  for X_batch, y_batch in train_loader:
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)

    optimizer.zero_grad()

    y_pred = model(X_batch)
    loss = criterion(y_pred, y_batch)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item() * X_batch.size(0)

  epoch_loss /= len(train_loader.dataset)
  print(f"Epoch {epoch+1}/{num_epochs} - train MSE: {epoch_loss:.6f}")


Device: cuda
7
Epoch 1/20 - train MSE: 0.173418
Epoch 2/20 - train MSE: 0.030708
Epoch 3/20 - train MSE: 0.022164
Epoch 4/20 - train MSE: 0.014709
Epoch 5/20 - train MSE: 0.009305
Epoch 6/20 - train MSE: 0.007906
Epoch 7/20 - train MSE: 0.007046
Epoch 8/20 - train MSE: 0.006411
Epoch 9/20 - train MSE: 0.005874
Epoch 10/20 - train MSE: 0.005489
Epoch 11/20 - train MSE: 0.005387
Epoch 12/20 - train MSE: 0.005039
Epoch 13/20 - train MSE: 0.004748
Epoch 14/20 - train MSE: 0.004617
Epoch 15/20 - train MSE: 0.004634
Epoch 16/20 - train MSE: 0.004536
Epoch 17/20 - train MSE: 0.004254
Epoch 18/20 - train MSE: 0.004124
Epoch 19/20 - train MSE: 0.003865
Epoch 20/20 - train MSE: 0.003744


# Evaluating the model

In [ ]:
model.eval()
test_loss = 0.0

with torch.no_grad():
  for X_batch, y_batch in test_loader:
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)

    y_pred = model(X_batch)
    loss = criterion(y_pred, y_batch)

    test_loss += loss.item() * X_batch.size(0)

test_loss /= len(test_loader.dataset)
print(f"Test MSE: {test_loss:.6f}")

Test MSE: 0.040213


In [ ]:
model.eval()

all_preds_scaled = []
all_targets_scaled = []

with torch.no_grad():
  for X_batch, y_batch in test_loader:
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)

    y_pred = model(X_batch)

    all_preds_scaled.append(y_pred.cpu().numpy())
    all_targets_scaled.append(y_batch.cpu().numpy())

y_pred_scaled = np.concatenate(all_preds_scaled, axis=0)
y_true_scaled = np.concatenate(all_targets_scaled, axis=0)

print("Shapes (scaled):", y_pred_scaled.shape, y_true_scaled.shape)


Shapes (scaled): (79, 1) (79, 1)


# Transforming the prediction back to USD

In [ ]:
n_test = y_pred_scaled.shape[0]

# Temporary array with len(cols_price)
temp_pred = np.zeros((n_test, len(cols_price)))
temp_true = np.zeros((n_test, len(cols_price)))

# Putting Close (scaled) in the correct column
temp_pred[:, close_index] = y_pred_scaled[:, 0]
temp_true[:, close_index] = y_true_scaled[:, 0]

# Inverting the scale
temp_pred_inv = scaler_price.inverse_transform(temp_pred)
temp_true_inv = scaler_price.inverse_transform(temp_true)

# Getting "Close" column in the original scale
y_pred_real = temp_pred_inv[:, close_index]
y_true_real = temp_true_inv[:, close_index]

print("Shapes (real):", y_pred_real.shape, y_true_real.shape)
print("5 first predicted values:", y_pred_real[:5])
print("5 first real values:", y_true_real[:5])

Shapes (real): (79,) (79,)
5 first predicted values: [219.50538773 220.25793192 221.11223499 222.32854766 223.99757539]
5 first real values: [216.27000236 226.16999952 228.13000176 229.80999378 230.28999574]
